In [ ]:
import pickle   
import numpy as np
import pandas as pd

import UtilityFunctions as uf

from tensorflow.keras.models import load_model
from sklearn import metrics

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

- List of Dataset for Novice and Expert pairs, and different sampling time 

        ''      stands for dt = 0.02 (default)

        'step2' stands for dt = 0.04

        'step4' stands for dt = 0.08
        
Datasets available at [ExplainedDecisions](https://osf.io/wgk8e/?view_only=8aec18499ed8457cb296032545963542) public repository.

In [ ]:
Dataset_list = ["Datasets/DatasetFileMultiClassPred_BothHerders_WrtGoal_Extended", 
                "Datasets/DatasetFileMultiClassPred_BothHerders_WrtGoal_Extended_step2",
                "Datasets/DatasetFileMultiClassPred_BothHerders_WrtGoal_Extended_step4",
                "Datasets/DatasetFileMultiClassPred_BothHerders_Expert_WrtGoal_Extended",
                "Datasets/DatasetFileMultiClassPred_BothHerders_Expert_WrtGoal_Extended_step2",
                "Datasets/DatasetFileMultiClassPred_BothHerders_Expert_WrtGoal_Extended_step4"]

- Input features (or state variables) to be extracted from the Dataset loaded

In [ ]:
Labels = ['h_t0 rel dist', 'h_t1 rel dist', 'h_t2 rel dist', 'h_t3 rel dist', 'h_t0 rel angle', 'h_t1 rel angle', 'h_t2 rel angle', 'h_t3 rel angle', 
          'h_goal rel dist', 't0_goal rel dist', 't1_goal rel dist', 't2_goal rel dist', 't3_goal rel dist',
          'h vel_r' , 't0 vel_r' , 't1 vel_r' , 't2 vel_r' ,  't3 vel_r' , 
          'h acc_r', 't0 acc_r', 't1 acc_r', 't2 acc_r', 't3 acc_r', 
          'h_goal_th', 't0_goal_th', 't1_goal_th', 't2_goal_th', 't3_goal_th', 
          'h_dir_motion', 't0_dir_motion', 't1_dir_motion', 't2_dir_motion', 't3_dir_motion',
          'h_h1 rel dist', 'h_h1 rel angle', 'h1_goal rel dist', 'h1 vel_r', 'h1 acc_r',
          'h1_goal_th', 'h1_dir_motion', 'h1_t0 rel dist', 'h1_t1 rel dist', 'h1_t2 rel dist', 'h1_t3 rel dist', 
          'h1_t0 rel angle', 'h1_t1 rel angle', 'h1_t2 rel angle', 'h1_t3 rel angle','Label']

Labels.insert(0,"Herder_id")
Labels.insert(1,"Trial_id")

- List of decision horizons

In [ ]:
look_forward_list = [1, 8, 16, 32]

- List of ANN trained for each combination of sampling time ('step') and decision horizon ('look_forward')

In [ ]:
directory = './checkpoint/FinalModels/'

Novice_step1 = [directory + '02032022/TrainTestSets_Novice_step1_thor1', 
                directory + '02032022/TrainTestSets_Novice_step1_thor8',
                directory + '04032022/TrainTestSets_Novice_step1_thor16',
                directory + '04032022/TrainTestSets_Novice_step1_thor32']

Novice_step2 = [directory + '28022022/TrainTestSets_Novice_step2_thor1', 
                directory + '01032022/TrainTestSets_Novice_step2_thor8',
                directory + '26022022/TrainTestSets_Novice_step2_thor16',
                directory + '28022022/TrainTestSets_Novice_step2_thor32']

Novice_step4 = [directory + '05032022/TrainTestSets_Novice_step4_thor1', 
                directory + '06032022/TrainTestSets_Novice_step4_thor8',
                directory + '06032022/TrainTestSets_Novice_step4_thor16',
                directory + '06032022/TrainTestSets_Novice_step4_thor32']

Expert_step1 = [directory + '04032022/TrainTestSets_Expert_step1_thor1', 
                directory + '04032022/TrainTestSets_Expert_step1_thor8',
                directory + '04032022/TrainTestSets_Expert_step1_thor16',
                directory + '05032022/TrainTestSets_Expert_step1_thor32']

Expert_step2 = [directory + '20022022/TrainTestSets_Expert_step2_thor1', 
                directory + '01032022/TrainTestSets_Expert_step2_thor8',
                directory + '05032022/TrainTestSets_Expert_step2_thor16',
                directory + '28022022/TrainTestSets_Expert_step2_thor32']

Expert_step4 = [directory + '06032022/TrainTestSets_Expert_step4_thor1', 
                directory + '06032022/TrainTestSets_Expert_step4_thor8',
                directory + '06032022/TrainTestSets_Expert_step4_thor16',
                directory + '07032022/TrainTestSets_Expert_step4_thor32']

file_set_list = [Novice_step1, Novice_step2, Novice_step4, Expert_step1, Expert_step2, Expert_step4]

In [ ]:
Novice_step1_model = [directory + '02032022/02032022001', 
                directory + '02032022/02032022002',
                directory + '04032022/04032022001',
                directory + '04032022/04032022002']

Novice_step2_model = [directory + '28022022/28022022004', 
                directory + '01032022/01032022001',
                directory + '26022022/26022022001',
                directory + '28022022/28022022001']

Novice_step4_model = [directory + '05032022/05032022003', 
                directory + '06032022/06032022001',
                directory + '06032022/06032022002',
                directory + '06032022/06032022003']

Expert_step1_model = [directory + '04032022/04032022003', 
                directory + '04032022/04032022004',
                directory + '04032022/04032022005',
                directory + '05032022/05032022001']

Expert_step2_model = [directory + '20022022/20022022003', 
                directory + '01032022/01032022002',
                directory + '05032022/05032022002',
                directory + '28022022/28022022002']

Expert_step4_model = [directory + '06032022/06032022004', 
                directory + '06032022/06032022005',
                directory + '06032022/06032022006',
                directory + '07032022/07032022001']

file_model_list = [Novice_step1_model, Novice_step2_model, Novice_step4_model, 
                   Expert_step1_model, Expert_step2_model, Expert_step4_model]

# Evaluate performance for each ANN trained, listed above

In [ ]:
for j in range(len(Dataset_list)):
    
    # load the dataset    
    file = open(Dataset_list[j],"rb")   
    Dataset_full_df = pickle.load(file)
    
    # extract the input features referred by Labels
    Dataset_df = Dataset_full_df[Labels]
    n_features = len(Dataset_df.columns) - 3
    print("\n there are ", n_features," features!")

    Dataset = Dataset_df.values
  
    for i in range(4):
        print(look_forward_list[i])
        print(file_set_list[j][i])
        
        # Create the sequences of features and target outputs from the dataset
        look_forward = look_forward_list[i]
        look_back = 25
        
        sequences, sequences_labels, targets = [],[],[]

        herders_tot = int(max(Dataset[:,0])) + 1
        trial_tot = int(max(Dataset[:,1])) + 1

        for herder_id in range(herders_tot):
            for trial_id in range(trial_tot):
                Dtst = Dataset_df[(Dataset_df["Herder_id"]==herder_id) & (Dataset_df["Trial_id"]==trial_id)].values[:,2:]
                seq, tar, seq_lbl = uf.create_dataset(Dtst, look_back, look_forward)
                sequences = sequences + seq
                targets = targets + tar
                sequences_labels = sequences_labels + seq_lbl

        sequences_array = np.array(sequences)
        targets_array = np.array(targets)
        sequences_labels_array = np.array(sequences_labels)
         
        # From the total available samples select the ones used for training and test during the training phase
        file_to_open = open(file_set_list[j][i],"rb")
        indexes_data = pickle.load(file_to_open)
        file_to_open.close()

        type_index = indexes_data[0]
        train_index = indexes_data[1]
        test_index = indexes_data[2]

        X_senior, y_senior, Z_senior = sequences_array[type_index], targets_array[type_index], sequences_labels_array[type_index]
        X_test = X_senior[test_index]
        y_test = y_senior[test_index]
        Z_test = Z_senior[test_index]

        targets_labels_array = uf.checkSamplesType(Z_test)
        dummies_train = pd.get_dummies(y_train)

        train = X_train
        train_target = dummies_train.values    

        dummies_test = pd.get_dummies(y_test)

        test = X_test
        test_target = dummies_test.values

        test_set = X_test
        test_set_target = dummies_test.values
        
        
        # Load the trained ANN         
        file_model = file_model_list[j][i]
        
        print(file_model)
        
        model = load_model(file_model)
        
        # Use the trained ANN on the test set        
        test_preds = model.predict(test_set)
        
        # Compute metrics for the trained ANN 
        predicted_classes = np.argmax(test_preds,axis=1)
        expected_classes = np.argmax(test_set_target,axis=1)
        correct = metrics.accuracy_score(expected_classes,predicted_classes)
        
        precision_recall_f1 = metrics.precision_recall_fscore_support(expected_classes,predicted_classes)

        precision, recall, f1 = 0, 0, 0

        for i in range (5): 
            precision = precision + precision_recall_f1[0][i]
            recall = recall + precision_recall_f1[1][i]
            f1 = f1 + precision_recall_f1[2][i]

        kappascore = metrics.cohen_kappa_score(expected_classes, predicted_classes)

        print("\n------ Accuracy: %.2f%%" % (correct*100))

        print('\n\n')